# Programming your solver

In this notebook, we will look at some of the more advanced capabilities Firedrake has for configuring and developing preconditioners.

As our prototypical example, we will consider the Stokes equations. Find $(u, p) \in V \times Q \subset (H^1)^d \times L^2$ such that

$$
\begin{align}
  \nu\int_\Omega \nabla u : \nabla v\,\mathrm{d}x - \int_\Omega p
  \nabla \cdot v\,\mathrm{d}x
  &= \int_\Omega f \cdot v\,\mathrm{d}x, \\
  -\int_\Omega \nabla \cdot u q \,\mathrm{d}x&= 0.
\end{align}
$$
for all $(v, q) \in V \times Q$. Where $\nu$ is the viscosity, and we have not yet introduced boundary conditions.

We're going to need a domain to solve this problem on, along with appropriate finite elements for the velocity and pressure spaces. Here we'll use the Taylor-Hood pair, although other options are available.

We start by importing functionality from the Firedrake interface, defining a mesh, and building the function spaces on it.

In [1]:
from firedrake import *

mesh = UnitSquareMesh(64, 64)

V = VectorFunctionSpace(mesh, "CG", 2)
Q = FunctionSpace(mesh, "CG", 1)
W = V*Q

## The variational problem

We will choose a viscosity that does not depend on the solution, and hence solve a linear set of equations. We therefore need some trial and test functions to build a symbolic representation of the problem, along with a function to hold the discrete solution.

In [ ]:
u, p = TrialFunctions(W)
v, q = TestFunctions(W)

nu = Constant(0.001)
a = inner(grad(u), grad(v))*dx - p*div(v)*dx - div(u)*q*dx
L = inner(f, v)*dx

w_h = Function(W)

We can go ahead and solve this problem, by calling

In [2]:
solve(a == L, w_h)

NameError: name 'a' is not defined

## How is the solver configured?

Firedrake relies on PETSc to do the heavy lifting for algebraic solves. If you don't provide any parameters, we just inherit the defaults from PETSc. Of course, we can change this, and probably want to so that we pick an appropriate solver for our problem.

Configuring the solver is done by providing _options_ to the solver in the form of a dictionary of solver parameters. These are passed, essentially unmodified, to PETSc which uses them to set up the solver.

So, to switch from whatever the default happened to be to a direct factorisation, we do the following.

In [3]:
parameters={"mat_type": "aij", "ksp_type": "preonly", "pc_type": "lu"}
solve(a == L, w_h, solver_parameters=parameters)

NameError: name 'a' is not defined

### A more scalable approach

LU factorisation is a reasonable choice for small to medium problems (especially in two dimensions). But if we want to eventually solve high resolution problems, we're going to need to use an iterative method with an appropriate preconditioner. We'll look options based around block preconditioners first, and then a monolithic multigrid approach.

### Programming with options

The number of options we're going to use is going to start to grow quite quickly. It is worthwhile, therefore, taking a little time to discuss the "programming language" PETSc exposes through its options database.

PETSc objects (such as solvers) are configurable at runtime, and the primary mechanism for doing so is through PETSc's _options database_. A PETSc object examines the database, and configures itself, when `setFromOptions` is called on it. For the solvers in Firedrake, this is arranged to happen when you call `solve`. The "programming language" for the options consists of two operations:

1. Value assignment -foo_type bar (equivalently in dict syntax `{"foo_type": "bar"}`)
2. String concatenation

The former says "set the `foo_type` key in the database to value `bar`", the latter is used, in combination with object _prefixes_ to configure multiple objects _of the same type_ in different ways.

